In [2]:
import requests
import json
import re
from newspaper import Article
from urllib.request import Request, urlopen
from bs4 import BeautifulSoup as soup
import time
import datetime
import pandas as pd
from newspaper.article import ArticleException, ArticleDownloadState

In [ ]:
search_lst = ["apple", "microsoft", "gamestop", "mercedes-benz", "tesla",
               "toyota", "nike", "adidas", "walmart", "target", "the-home-depot",
               "best-buy", "costco", "hyatt-hotels", "marriott-hotels-resorts", "verizon", 
               "att", "citigroup", "jpmorgan-chase", "wells-fargo", "mcdonalds",
               "berkshire-hathaway", "delta-air-lines", "state-farm", "starbucks",
               "allstate-insurance", "cvs-health", "rogers-communications", "cineplex"]
check_lst = ["Apple", "Microsoft", "GameStop", "Mercedes", "Tesla",
              "Toyota", "Nike", "Adidas", "Walmart", "Target", "Home Depot",
              "Best Buy", "Costco", "Hyatt", "Mariott", "Verizon", 
              "AT&T", "Citi", "JPMorgan", "Wells", "McDonald", "Berkshire", "Delta", 
              "State Farm", "Starbucks", "Allstate", "CVS", "Rogers", "Cineplex"]
links = []

for idx, search in enumerate(search_lst):
    for page_number in range(1,15): 
        page_number=str(page_number)
        
        if search == 'cineplex' or search == 'rogers-communications':
            base_url_huff = 'https://www.huffingtonpost.ca/news/' + search + '/?page='
        else:
            base_url_huff = 'https://www.huffpost.com/impact/topic/' + search + '?page='
        
        r_huff = Request(base_url_huff + page_number, headers={'User-Agent': 'Chrome/90.0.4430.212'})
        webpage = urlopen(r_huff).read()
        page_soup = soup(webpage,"html.parser")
        page_class = page_soup.findAll("a", {"class": "card__headline card__headline--long"})
        
        for link in page_class:
            links.append([search, check_lst[idx], link.get('href')])
        
        time.sleep(5)
    time.sleep(30)

In [ ]:
df_links = pd.DataFrame(links, columns=['Search', 'Check', 'Link'])
df_links.to_csv('../Data/huff_links.csv', index=False, header=True)

In [20]:
df_huff = pd.DataFrame()

for i in range(0, len(links)): 
    try:
        article_huff = Article(links[i][2])
        article_huff.download()
        article_huff.parse()
        article_huff.nlp()

        article_info_huff = {'company': [links[i][0]],
                             'link': [links[i][2]],
                             'title' : [article_huff.title], 
                             'description': [article_huff.meta_description],
                             'nlp_summary': [article_huff.summary],
                             'publish date': [datetime.datetime.date(article_huff.publish_date)]}

        temp_df = pd.DataFrame(article_info_huff)
        df_huff = df_huff.append(temp_df)

        time.sleep(2)
    except:
        article_info_huff = {'company': [links[i][0]],
                             'link': [links[i][2]], 
                             'title': ["null"], 
                             'description': ["null"], 
                             'nlp_summary': ["null"],
                             'publish date': ["null"]}

        temp_df = pd.DataFrame(article_info_huff)
        df_huff = df_huff.append(temp_df)
        continue

In [23]:
df_huff.to_csv('../Data/huffpost_articles.csv', index=False, header=True)